In [ ]:
import numpy as np
from pymoo.util.misc import stack
from pymoo.model.problem import Problem

#pd.options.display.precision = 2


class MyProblem(Problem):

    def __init__(self, hit_pts=np.array([]), miss_pts=np.array([])):
        super().__init__(n_var=6,
                         n_obj=2,
                         n_constr=0,
                         xl=np.array([-350,-350,-350,-350,-350,-350]),
                         xu=np.array([350,350,350,350,350,350]))
        
        self.hit_pts = hit_pts
        self.miss_pts = miss_pts

    def _evaluate(self, x, out, *args, **kwargs):
        
        f1 = self.hitcost(x)
        f2 = self.misscost(x)

                      
        out["F"] = np.column_stack([f1, f2])
    
        
    #all_linepts = np.tile(linepts.T, (3,1,1))
    def ptsfromline(self, pts, linepts):

        #dlist = []
        a = linepts.T[:,:1]
        b = linepts.T[:,1:]

        #print(np.linalg.norm(b-a, axis=0))

        d = np.linalg.norm(np.cross(pts - a, pts-b, axis=0), axis=0) / np.linalg.norm(b-a, axis=0)

        return d
    
    def hitcost (self, x): 
    
        costs = []

        for line in x:
            cubeLength = 50
            inside = cubeLength/2*np.sqrt(3)

            bound = (cubeLength + inside) / 2

            p = line[:3]
            v = line[3:]

            linepts = v * np.mgrid[-.5:.5:2j][:, np.newaxis] + p

            hitlist = self.ptsfromline(self.hit_pts, linepts)

            #return sum([d**2 for d in hitlist]) + sum([inside**4/d**2 for d in misslist])

            linecost =  sum([1/(1 + np.exp(-.1*(d-42))) for d in hitlist]) 
            costs.append(linecost)
            
        return costs

    def misscost (self, x): 

        costs = []

        for line in x:
            cubeLength = 50
            inside = cubeLength/2*np.sqrt(3)

            bound = (cubeLength + inside) / 2

            p = line[:3]
            v = line[3:]

            linepts = v * np.mgrid[-.5:.5:2j][:, np.newaxis] + p

            misslist = self.ptsfromline(self.miss_pts, linepts)

            #return sum([d**2 for d in hitlist]) + sum([inside**4/d**2 for d in misslist])

            linecost =  sum([1/(1 + np.exp(.1*(d-25))) for d in misslist])
            costs.append(linecost)
            
        return costs